 **Imdb example code**

In [2]:
!pip install tensorflow-hub
!pip install tensorflow-datasets
!pip install tf-keras

In [3]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tf_keras as keras

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.18.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


In [4]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.8R85SU_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.8R85SU_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.8R85SU_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [7]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [8]:
model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 59s 2s/step - loss: 0.6577 - accuracy: 0.5244 - val_loss: 0.6163 - val_accuracy: 0.5620
Epoch 2/10
30/30 [==============================] - 45s 1s/step - loss: 0.5569 - accuracy: 0.6594 - val_loss: 0.5138 - val_accuracy: 0.7235
Epoch 3/10
30/30 [==============================] - 45s 1s/step - loss: 0.4248 - accuracy: 0.8097 - val_loss: 0.4083 - val_accuracy: 0.8182
Epoch 4/10
30/30 [==============================] - 45s 1s/step - loss: 0.3070 - accuracy: 0.8801 - val_loss: 0.3487 - val_accuracy: 0.8367
Epoch 5/10
30/30 [==============================] - 47s 2s/step - loss: 0.2267 - accuracy: 0.9159 - val_loss: 0.3188 - val_accuracy: 0.8614
Epoch 6/10
30/30 [==============================] - 46s 2s/step - loss: 0.1700 - accuracy: 0.9434 - val_loss: 0.3055 - val_accuracy: 0.8680
Epoch 7/10
30/30 [==============================] - 51s 2s/step - loss: 0.1274 - accuracy: 0.9633 - val_loss: 0.3024 - val_accuracy: 0.8727
Epoch 8/10
30/30 [==

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 12s - loss: 0.3519 - accuracy: 0.8555 - 12s/epoch - 253ms/step
loss: 0.352
accuracy: 0.856


# Modified Code

In [29]:
# Load IMDB dataset and split into training (60%), validation (40%), and testing (100%)
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [30]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [31]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [32]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [34]:
# Experiment with different model configurations
model = keras.Sequential()
model.add(hub_layer)

In [38]:
# Three hidden layers with tanh activation function
model.add(keras.layers.Dense(64, activation='tanh'))
model.add(keras.layers.Dense(32, activation='tanh'))
model.add(keras.layers.Dense(16, activation='tanh'))

In [40]:
# Dropout for regularization
model.add(keras.layers.Dropout(0.5))

In [42]:
# Output layer
model.add(keras.layers.Dense(1))

In [43]:
# Model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 64)                3264      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 16)                528       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                

In [45]:
model.compile(
    optimizer='rmsProp',
    loss=tf.keras.losses.MeanSquaredError(),  # Changed from binary_crossentropy to mse
    metrics=['accuracy']
)

In [46]:
# Train the model
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=10,
    validation_data=validation_data.batch(512),
    verbose=1
)

Epoch 1/10
30/30 [==============================] - 26s 802ms/step - loss: 0.3245 - accuracy: 0.5595 - val_loss: 0.2113 - val_accuracy: 0.6566
Epoch 2/10
30/30 [==============================] - 36s 1s/step - loss: 0.2332 - accuracy: 0.6564 - val_loss: 0.2045 - val_accuracy: 0.6711
Epoch 3/10
30/30 [==============================] - 23s 769ms/step - loss: 0.2027 - accuracy: 0.7027 - val_loss: 0.2336 - val_accuracy: 0.6418
Epoch 4/10
30/30 [==============================] - 25s 848ms/step - loss: 0.1959 - accuracy: 0.7225 - val_loss: 0.1567 - val_accuracy: 0.7784
Epoch 5/10
30/30 [==============================] - 24s 810ms/step - loss: 0.1785 - accuracy: 0.7462 - val_loss: 0.1422 - val_accuracy: 0.8116
Epoch 6/10
30/30 [==============================] - 24s 813ms/step - loss: 0.1686 - accuracy: 0.7640 - val_loss: 0.1703 - val_accuracy: 0.7466
Epoch 7/10
30/30 [==============================] - 25s 837ms/step - loss: 0.1567 - accuracy: 0.7893 - val_loss: 0.1507 - val_accuracy: 0.7895
Ep

In [47]:
# Evaluate the model
results = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 11s - loss: 0.1269 - accuracy: 0.8260 - 11s/epoch - 225ms/step


In [48]:
# Print results
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 0.127
accuracy: 0.826


Model Configuration:

Embedding Layer: Pre-trained TensorFlow Hub embedding (nnlm-en-dim50/2)

Hidden Layers: Three layers with 64, 32, and 16 units, using tanh activation.

Regularization: Dropout layer with a rate of 0.5.

Loss Function: Mean Squared Error (MSE) instead of Binary Crossentropy.

Optimizer: rmsProp Optimizer.

Batch Size: 512

Epochs: 10

Performance Metrics:

Training Accuracy: 83.29%

Validation Accuracy: 83.12%

Test Accuracy: 82.6%

Test Loss: 0.127

The study and conclusion point to the balanced model showing consistent performance over all three datasets: training, validation, and test. Unlike previous systems beset with overfitting, this particular model has shown better generalization and consistent performance across datasets. The test accuracy dropped as the training accuracy peaked at 99% in those more traditional designs.

Dropout regularization and the tanh activation function helped the model to lower its overfitting capacity while preserving its great degree of predictive performance. Moreover, using optimization allowed mean squared error to substitute binary_crossentropy, so improving the error minimizing procedure.